# Package

In [ ]:
# Basic
import numpy as np
import os

import tensorflow as tf
from tensorflow.keras.optimizers import Adam

# Custom
from Utils.utils import *
from Utils.model import *
from Utils.path import *

# Load Data

In [ ]:
data_path = os.path.join(Paths.DATA_PATH, "Subscription_Boxes.jsonl.gz")

In [ ]:
df = getDF(data_path)

In [14]:
df.head()

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,1.0,USELESS,Absolutely useless nonsense and a complete was...,[],B07G584SHG,B09WC47S3V,AEMJ2EG5ODOCYUTI54NBXZHDJGSQ,1602133857705,2,True
1,2.0,Manufactured where?,"With a couple of the items, I wasn't quite sur...",[],B07QL1JRCN,B07QL1JRCN,AEEJBFZKUBEEMBZUZJV4UHFVEEBQ,1609110735433,20,True
2,1.0,Little bang for your buck.,Two SMALL stuffed animals and 2 little bags of...,[],B07RBYJN37,B08N5QKX1Y,AGSVZNZBTSGQBKZDZTQYEZHGDPCQ,1609937315319,4,True
3,5.0,New favorite box,"Although I don’t remember signing up for this,...",[],B07KM6T8GV,B07KM6T8GV,AFDERNB6BIR3U2DOR3S2KX7KJJXQ,1616156351887,1,True
4,5.0,Coctique,I loved every thing and could use it all. Thin...,[],B07NVL6TJG,B07NVKNVNM,AE6P2YJ6FKX332MD56GPJFSHXNJQ,1559533206066,0,True


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77071 entries, 0 to 77070
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   overall         77071 non-null  float64
 1   verified        77071 non-null  bool   
 2   reviewTime      77071 non-null  object 
 3   reviewerID      77071 non-null  object 
 4   asin            77071 non-null  object 
 5   style           36037 non-null  object 
 6   reviewerName    77044 non-null  object 
 7   reviewText      77060 non-null  object 
 8   summary         77061 non-null  object 
 9   unixReviewTime  77071 non-null  int64  
 10  vote            9620 non-null   object 
 11  image           1719 non-null   object 
dtypes: bool(1), float64(1), int64(1), object(9)
memory usage: 7.1+ MB


In [11]:
df, USER_LEN, ITEM_LEN = Load_data(df, "reviewerID", "asin", "overall", "reviewText")

<ipython-input-4-6c02a1d01fab>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns = {user: "user",


In [12]:
df.head()

,user,item,rating,text
0,1557,0,5.0,This worked really well for what I used it for...
1,4282,0,5.0,Fast cutting and good adheasive.
2,7415,0,5.0,Worked great for my lapping bench. I would li...
3,10602,1,4.0,As advertised
4,2574,1,5.0,seems like a pretty good value as opposed to b...


In [13]:
train_df, test_df = train_validation_test(df)

100%|██████████| 11041/11041 [00:46<00:00, 236.14it/s]

Train dataset shape: (57957, 4)
Test dataset shape: (19068, 4)
True
True


In [15]:
train_ds, validation_ds, test_ds = bert_roberta(train_df, test_df, batch_size = 2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


cuda


100%|██████████| 5521/5521 [03:46<00:00, 24.38it/s]


cuda


100%|██████████| 2662/2662 [02:27<00:00, 18.02it/s]


cuda


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 5521/5521 [03:37<00:00, 25.44it/s]


cuda


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 2662/2662 [02:19<00:00, 19.14it/s]


In [18]:
train_dict = {"train_user": train_ds["user"].values.astype(np.float32),
              "train_item": train_ds["item"].values.astype(np.float32),
              "train_user_bert": np.array(train_ds["user_bert"].tolist()),
              "train_item_bert": np.array(train_ds["item_bert"].tolist()),
              "train_user_roberta": np.array(train_ds["user_roberta"].tolist()),
              "train_item_roberta": np.array(train_ds["item_roberta"].tolist()),
              "train_y": train_ds["rating"].values.astype(np.float32)}

validation_dict = {"validation_user": validation_ds["user"].values.astype(np.float32),
              "validation_item": validation_ds["item"].values.astype(np.float32),
              "validation_user_bert": np.array(validation_ds["user_bert"].tolist()),
              "validation_item_bert": np.array(validation_ds["item_bert"].tolist()),
              "validation_user_roberta": np.array(validation_ds["user_roberta"].tolist()),
              "validation_item_roberta": np.array(validation_ds["item_roberta"].tolist()),
              "validation_y": validation_ds["rating"].values.astype(np.float32)}

test_dict = {"test_user": test_ds["user"].values.astype(np.float32),
              "test_item": test_ds["item"].values.astype(np.float32),
              "test_user_bert": np.array(test_ds["user_bert"].tolist()),
              "test_item_bert": np.array(test_ds["item_bert"].tolist()),
              "test_user_roberta": np.array(test_ds["user_roberta"].tolist()),
              "test_item_roberta": np.array(test_ds["item_roberta"].tolist()),
              "test_y": test_ds["rating"].values.astype(np.float32)}

In [19]:
train_tfds = tf.data.Dataset.from_tensor_slices((train_dict["train_user"], train_dict["train_item"],
                                               train_dict["train_user_bert"], train_dict["train_item_bert"],
                                               train_dict["train_user_roberta"], train_dict["train_item_roberta"],
                                               train_dict["train_y"]))
train_tfds = train_tfds.shuffle(1000).batch(1024)

validation_tfds = tf.data.Dataset.from_tensor_slices((validation_dict["validation_user"], validation_dict["validation_item"],
                                                    validation_dict["validation_user_bert"], validation_dict["validation_item_bert"],
                                                    validation_dict["validation_user_roberta"], validation_dict["validation_item_roberta"],
                                                    validation_dict["validation_y"]))
validation_tfds = validation_tfds.shuffle(1000).batch(512)

test_tfds = tf.data.Dataset.from_tensor_slices((test_dict["test_user"], test_dict["test_item"],
                                              test_dict["test_user_bert"], test_dict["test_item_bert"],
                                              test_dict["test_user_roberta"], test_dict["test_item_roberta"],
                                              test_dict["test_y"]))
test_tfds = test_tfds.shuffle(1000).batch(512)

# Initiation

In [24]:
EPOCHS = 100
LR = 0.001

loss_object = tf.keras.losses.MeanSquaredError()
optimizer = Adam(learning_rate = LR)

load_metrics()

# Model Train & Test

In [25]:
model = MFNR(USER_LEN, ITEM_LEN, 64)

In [26]:
loss_list = []
num = 0

for epoch in range(EPOCHS):
    trainer()
    validation()
    print(f"Epoch: {epoch}")
    print(f'train_loss: {(train_loss.result()):.3f}, validation_loss: {(validation_loss.result()):.3f}\n')

    loss_list.append(validation_loss.result())
    min_loss = np.min(loss_list)

    if validation_loss.result() <= min_loss:
        print("Model Save!")
        num = 0
        best_model = model
    else:
        num += 1
        print(f"num: {num}")
        if num == 5:
            print("early stopping")
            break

    train_loss.reset_states()
    validation_loss.reset_states()


Epoch: 0
train_loss: 2.080, validation_loss: 0.855

Model Save!

Epoch: 1
train_loss: 0.781, validation_loss: 0.787

Model Save!

Epoch: 2
train_loss: 0.634, validation_loss: 0.773

Model Save!

Epoch: 3
train_loss: 0.556, validation_loss: 0.845

num: 1

Epoch: 4
train_loss: 0.626, validation_loss: 0.910

num: 2

Epoch: 5
train_loss: 0.678, validation_loss: 0.830

num: 3

Epoch: 6
train_loss: 0.813, validation_loss: 0.970

num: 4

Epoch: 7
train_loss: 0.981, validation_loss: 0.791

num: 5
early stopping


In [27]:
tester()
print(f'Test_loss: {(test_loss.result()):.3f}')
print(f'Test_rmse: {(test_rmse.result()):.3f}')
print(f'Test_mae: {(test_mae.result()):.3f}')

Test_loss: 0.777
Test_rmse: 0.883
Test_mae: 0.605
